In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
DIR = './dataset/train/'

texts = {
    'id': [],
    'text-id': [],
    'text': [],
    'num-authors': [],
    'level': [],
    'change' : []
}


In [3]:
for level in range(1,4,1):
    for i in range(1,4201, 1):
        filename = f'problem-{i}.txt'
        with open('dataset\pan23-multi-author-analysis-dataset'+ str(level) + '\pan23-multi-author-analysis-dataset'+ str(level) + '-train/' + filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            lines = " ||| ".join(lines)

        json_filename = f'truth-problem-{i}.json'
        with open('dataset\pan23-multi-author-analysis-dataset'+ str(level) +'\pan23-multi-author-analysis-dataset'+ str(level) + '-train/' + json_filename, 'r', encoding='utf-8') as json_file:
            json_data = json.load(json_file)
            num_authors = json_data['authors']  # Get the maximum number of authors
            change_array = json_data['changes']

        if level == 1:
            id = 'easy-' + str(i)
        elif level == 2:
            id = 'medium-'+str(i)
        elif level == 3:
            id = 'hard-'+str(i)
        
        texts['id'].append(id)
        texts['text-id'].append(i)
        texts['text'].append(lines)
        texts['num-authors'].append(num_authors)
        texts['level'].append(level)
        texts['change'].append(change_array)

In [4]:
data = pd.DataFrame(texts)
data['text-length'] = data['text'].apply(lambda x: len(x))
data

,id,text-id,text,num-authors,level,change,text-length
0,easy-1,1,"I'm not arguing with you here, I'm simply tryi...",2,1,"[1, 0]",1076
1,easy-2,2,"It doesn't matter what community you are in, i...",2,1,"[0, 0, 0, 0, 0, 1]",1296
2,easy-3,3,"In general, be courteous to others. Debate/dis...",4,1,"[1, 1, 1, 1]",1247
3,easy-4,4,The parole and exchange system in the Civil Wa...,2,1,"[0, 0, 0, 1]",1350
4,easy-5,5,"In general, be courteous to others. Debate/dis...",4,1,"[1, 1, 1, 1]",1030
...,...,...,...,...,...,...,...
12595,hard-4196,4196,"They can ask, but they can't make it a require...",2,3,"[0, 1, 0, 0]",1148
12596,hard-4197,4197,"In the years preceding the Iranian Revolution,...",2,3,"[0, 0, 1]",2105
12597,hard-4198,4198,Does the order from the judge say the tenant d...,4,3,"[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",3920
12598,hard-4199,4199,The simple answer is that Western education sy...,2,3,"[0, 1, 1, 0, 0, 0, 0]",2608


In [5]:
data.describe()

,text-id,num-authors,level,text-length
count,12600.000000,12600.000000,12600.000000,12600.000000
mean,2100.500000,3.000000,2.000000,1830.334683
std,1212.483646,0.816529,0.816529,1405.847519
min,1.000000,2.000000,1.000000,1006.000000
25%,1050.750000,2.000000,1.000000,1144.000000
50%,2100.500000,3.000000,2.000000,1359.000000
75%,3150.250000,4.000000,3.000000,1857.000000
max,4200.000000,4.000000,3.000000,10265.000000


In [6]:
isolated_text = data.loc[(data['text-length'] >= data['text-length'].quantile(0.25)) & (data['text-length'] <= data['text-length'].quantile(0.75))]
isolated_text.shape

(6310, 7)

In [7]:
random.seed(39)
check_up_text = random.choice(isolated_text.loc[(isolated_text['num-authors'] == 3) & (isolated_text['level'] == 1), 'id'].tolist())
check_up_text

'easy-1190'

In [8]:
chosen_id = []
random.seed(39)
for i in range(32):
    texts_id = random.choice(isolated_text['id'].tolist())
    chosen_id.append(texts_id)

In [9]:
group_size = 4
for i in range(0, len(chosen_id), group_size):
    print("Version", (i // group_size) + 1)
    for j in range(group_size):
        if i + j < len(chosen_id):
            print("    {}".format(chosen_id[i + j]))
    print('-'*60)

Version 1
    easy-3312
    easy-4097
    medium-2473
    easy-389
------------------------------------------------------------
Version 2
    easy-3063
    easy-3470
    medium-2691
    hard-3582
------------------------------------------------------------
Version 3
    hard-1302
    easy-69
    hard-2459
    easy-4020
------------------------------------------------------------
Version 4
    hard-2869
    medium-1867
    hard-3322
    easy-2811
------------------------------------------------------------
Version 5
    easy-408
    medium-566
    hard-2847
    easy-1097
------------------------------------------------------------
Version 6
    medium-2076
    easy-117
    easy-1659
    medium-3270
------------------------------------------------------------
Version 7
    medium-1702
    medium-3240
    hard-3912
    easy-1471
------------------------------------------------------------
Version 8
    medium-2607
    hard-625
    hard-1559
    medium-682
---------------------------------

In [10]:
# The longest texts
data.loc[data['text-length'] == data.loc[data['id'].isin(chosen_id), 'text-length'].max()]

,id,text-id,text,num-authors,level,change,text-length
5901,medium-1702,1702,"Essentially, yes. They didn't have authority e...",2,2,"[0, 0, 0, 1]",1829
11543,hard-3144,3144,IIRC Stalin was fixated on the UK as a threat....,4,3,"[1, 1, 1, 0, 1, 0, 1]",1829


In [11]:
file_path = 'dataset_dataframe.csv'
# Save the DataFrame as CSV
data.to_csv(file_path, index=False)